In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_kidney_disease"
cohort = "GSE45980"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_kidney_disease"
in_cohort_dir = "../../input/GEO/Chronic_kidney_disease/GSE45980"

# Output paths
out_data_file = "../../output/preprocess/Chronic_kidney_disease/GSE45980.csv"
out_gene_data_file = "../../output/preprocess/Chronic_kidney_disease/gene_data/GSE45980.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE45980.csv"
json_path = "../../output/preprocess/Chronic_kidney_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"An integrative analysis of renal miRNA- and mRNA-expression signatures in progressive chronic kidney disease [discovery cohort]"
!Series_summary	"MicroRNAs (miRNAs) significantly contribute to chronic kidney disease (CKD) progression via regulating mRNA expression and abundance. However, their association with clinical outcome remains poorly understood. We performed large scale miRNA and mRNA expression profiling on cryo-cut renal biopsy sections from n=43 subjects. miRNAs differentiating stable and progressive cases were determined, and putative target mRNAs showing inversely correlated expression profiles were identified and further characterized. We found a downregulation of 7 miRNAs in the progressive phenotype, and an upregulation of 29 target mRNAs which are involved in inflammatory response, cell-cell-interaction, apoptosis, and intracellular signaling. Particularly a diminished expression of miR-206 in progressive disease correlated signif

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains mRNA expression profiling
# which is suitable for our study
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait: The key is 3, which refers to "clinical course: stable/progressive"
# For age: The key is 1, which contains "age (yrs): X"
# For gender: The key is 0, which contains "gender: male/female"
trait_row = 3
age_row = 1
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait (clinical course) to binary."""
    if pd.isna(value) or value is None:
        return None
    value = value.lower()
    if "clinical course:" in value:
        value = value.split("clinical course:")[1].strip()
    
    if "progressive" in value:
        return 1  # Progressive CKD
    elif "stable" in value:
        return 0  # Stable CKD
    else:
        return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if pd.isna(value) or value is None:
        return None
    try:
        if "age (yrs):" in value:
            age_str = value.split("age (yrs):")[1].strip()
            return float(age_str)
        return None
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if pd.isna(value) or value is None:
        return None
    value = value.lower()
    if "gender:" in value:
        value = value.split("gender:")[1].strip()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print("Preview of extracted clinical features:")
    print(preview_df(clinical_features))
    
    # Save the clinical features to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM1121040': [0.0, 72.0, 1.0], 'GSM1121041': [0.0, 20.0, 0.0], 'GSM1121042': [0.0, 64.0, 0.0], 'GSM1121043': [0.0, 17.0, 1.0], 'GSM1121044': [0.0, 46.0, 1.0], 'GSM1121045': [0.0, 55.0, 1.0], 'GSM1121046': [0.0, 74.0, 1.0], 'GSM1121047': [0.0, 49.0, 1.0], 'GSM1121048': [0.0, 20.0, 1.0], 'GSM1121049': [0.0, 42.0, 1.0], 'GSM1121050': [0.0, 73.0, 0.0], 'GSM1121051': [0.0, 63.0, 0.0], 'GSM1121052': [0.0, 33.0, 0.0], 'GSM1121053': [0.0, 74.0, 1.0], 'GSM1121054': [0.0, 24.0, 1.0], 'GSM1121055': [0.0, 45.0, 1.0], 'GSM1121056': [0.0, 70.0, 1.0], 'GSM1121057': [0.0, 60.0, 1.0], 'GSM1121058': [0.0, 67.0, 0.0], 'GSM1121059': [0.0, 31.0, 0.0], 'GSM1121060': [0.0, 53.0, 0.0], 'GSM1121061': [0.0, 67.0, 0.0], 'GSM1121062': [0.0, 22.0, 0.0], 'GSM1121063': [0.0, 54.0, 0.0], 'GSM1121064': [0.0, 40.0, 1.0], 'GSM1121065': [0.0, 38.0, 0.0], 'GSM1121066': [0.0, 19.0, 1.0], 'GSM1121067': [0.0, 28.0, 0.0], 'GSM1121068': [0.0, 65.0, 1.0], 'GSM1121069': [0.0, 74.0, 1.0],

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Assume gene data is available

# Extract gene data
try:
    # Extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # Print the first 20 gene/probe identifiers
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20].tolist())
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print(f"File path: {matrix_file}")
    print("Please check if the file exists and contains the expected markers.")


SOFT file: ../../input/GEO/Chronic_kidney_disease/GSE45980/GSE45980_family.soft.gz
Matrix file: ../../input/GEO/Chronic_kidney_disease/GSE45980/GSE45980_series_matrix.txt.gz
Gene data shape: (9665, 43)
First 20 gene/probe identifiers:
['A_23_P100001', 'A_23_P100240', 'A_23_P100315', 'A_23_P100326', 'A_23_P100355', 'A_23_P100392', 'A_23_P100486', 'A_23_P100501', 'A_23_P100660', 'A_23_P100704', 'A_23_P100764', 'A_23_P100963', 'A_23_P101111', 'A_23_P101332', 'A_23_P10135', 'A_23_P101407', 'A_23_P101480', 'A_23_P101516', 'A_23_P101532', 'A_23_P101551']


### Step 4: Gene Identifier Review

In [5]:
# The identifiers like 'A_23_P100001' are probe IDs from Agilent microarrays
# They are not human gene symbols and require mapping to gene symbols
# These are specific probe identifiers used in Agilent microarray platforms

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
# Based on the output, 'ID' and 'GENE_SYMBOL' are likely the mapping columns we need
gene_id_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

if gene_id_col in gene_annotation.columns and gene_symbol_col in gene_annotation.columns:
    print(f"\nSample mappings from '{gene_id_col}' to '{gene_symbol_col}':")
    sample_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].head(10)
    print(sample_mappings)
    
    # Check for non-null mappings to confirm data quality
    non_null_mappings = gene_annotation[[gene_id_col, gene_symbol_col]].dropna(subset=[gene_symbol_col])
    print(f"\nNumber of probes with gene symbol mappings: {len(non_null_mappings)}")
    print(f"Sample of valid mappings:")
    print(non_null_mappings.head(5))
else:
    print("Required mapping columns not found in the annotation data.")



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the columns for gene identifier and gene symbol mapping
probe_id_col = 'ID'  # This contains identifiers like 'A_23_P100001' matching the gene expression data
gene_symbol_col = 'GENE_SYMBOL'  # This contains the gene symbols we need

print(f"Using mapping from '{probe_id_col}' to '{gene_symbol_col}'")

# 2. Extract the gene mapping dataframe with the two relevant columns
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5].tolist())

# Show the first few values for a few genes to verify the data
print("\nPreview of gene expression data after mapping:")
print(gene_data.iloc[:5, :5])


Using mapping from 'ID' to 'GENE_SYMBOL'
Gene mapping dataframe shape: (29833, 2)
Sample of gene mapping:
              ID     Gene
11  A_23_P100001  FAM174B
12  A_23_P100022     SV2B
13  A_23_P100056   RBPMS2
14  A_23_P100074     AVEN
15  A_23_P100127    CASC5
Gene expression data shape after mapping: (8182, 43)
First 5 gene symbols after mapping:
['A1BG', 'A4GALT', 'AAAS', 'AACS', 'AADACL3']

Preview of gene expression data after mapping:
         GSM1121040  GSM1121041  GSM1121042  GSM1121043  GSM1121044
Gene                                                               
A1BG          5.819       6.883       5.939       5.627       5.884
A4GALT       10.260       9.439       9.304       9.026       8.972
AAAS          6.096       6.338       6.158       5.887       5.926
AACS          6.395       6.298       6.431       6.227       5.974
AADACL3       6.082       6.453       5.749       5.782       6.057


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load clinical data from previously saved file
try:
    # Try to load the clinical data from the file saved in Step 2
    selected_clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data from {out_clinical_data_file}")
except FileNotFoundError:
    # If not available, re-extract it using the correct row indices from Step 2
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    trait_row = 3  # Correct row index from Step 2
    age_row = 1
    gender_row = 0
    
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data extracted and saved to {out_clinical_data_file}")

print(f"Clinical data shape: {selected_clinical_df.shape}")

# 3. Link the clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
print(linked_data.iloc[:5, :5])

# Rename the clinical columns to match expected names
linked_data = linked_data.rename(columns={0: trait, 1: "Age", 2: "Gender"})
print("Linked data after renaming columns:")
print(linked_data.iloc[:5, :5])

# 4. Handle missing values in the linked data
print("\nHandling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine if the trait and demographic features are biased
print("\nChecking for bias in trait and demographic features...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation and save relevant information
print("\nConducting final quality validation...")
is_gene_available = True  # We've confirmed gene data is available in previous steps
is_trait_available = True  # We've confirmed trait data is available in previous steps

note = "This dataset contains gene expression data from kidney biopsies. It classifies samples based on clinical course (stable or progressive chronic kidney disease)."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved as dataset is not usable for the current trait study.")


Normalizing gene symbols...
Gene data shape after normalization: (8116, 43)


Normalized gene data saved to ../../output/preprocess/Chronic_kidney_disease/gene_data/GSE45980.csv
Loaded clinical data from ../../output/preprocess/Chronic_kidney_disease/clinical_data/GSE45980.csv
Clinical data shape: (3, 43)

Linking clinical and genetic data...
Linked data shape: (43, 8119)
Linked data preview (first 5 rows, first 5 columns):
              0     1    2   A1BG  A4GALT
GSM1121040  0.0  72.0  1.0  5.819  10.260
GSM1121041  0.0  20.0  0.0  6.883   9.439
GSM1121042  0.0  64.0  0.0  5.939   9.304
GSM1121043  0.0  17.0  1.0  5.627   9.026
GSM1121044  0.0  46.0  1.0  5.884   8.972
Linked data after renaming columns:
            Chronic_kidney_disease   Age  Gender   A1BG  A4GALT
GSM1121040                     0.0  72.0     1.0  5.819  10.260
GSM1121041                     0.0  20.0     0.0  6.883   9.439
GSM1121042                     0.0  64.0     0.0  5.939   9.304
GSM1121043                     0.0  17.0     1.0  5.627   9.026
GSM1121044                     0.0  46.0  

Linked data shape after handling missing values: (43, 8119)

Checking for bias in trait and demographic features...
For the feature 'Chronic_kidney_disease', the least common label is '1.0' with 12 occurrences. This represents 27.91% of the dataset.
The distribution of the feature 'Chronic_kidney_disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 36.0
  50% (Median): 55.0
  75%: 65.0
Min: 17.0
Max: 74.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 15 occurrences. This represents 34.88% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Conducting final quality validation...


Linked data saved to ../../output/preprocess/Chronic_kidney_disease/GSE45980.csv
